# How to train keras_OCR with ISIS ionograms

## 1. Prepare dataset
I selected randomly 60 ionograms of ISIS. I than created a csv file with annotations. Here's how it looks.

In [55]:
import pandas as pd

ann = pd.read_csv('L:/DATA/ISIS/keras_ocr/annotations/global_annotations.csv')
ann.head()

,image_path,x_min,y_min,x_max,y_max,text_content
0,1.png,316,361,601,402,36 00 69 111 0354 35
1,2.png,457,363,746,405,36 00 69 111 0245 04
2,3.png,481,369,767,403,36 00 69 101 0245 31
3,4.png,510,366,793,402,36 00 69 111 0245 58
4,5.png,492,368,774,402,36 00 69 111 0246 25


The columns (x_min, y_min) and (x_max, y_max) define the top left corner and the bottom right corner of a box around the metadata. I found those values manually in paint. It took me around 1h30 to do this.

## 2. Split the dataset
I need to create three subsets : training, validation and test sets. Since I only have 60 images, I'll use the 70-15-15 ratio to create the subsets. The first step is to shuffle the dataset randomly.

In [56]:
ann = ann.sample(frac = 1)
ann.reset_index(drop=True, inplace=True)
ann.head()

,image_path,x_min,y_min,x_max,y_max,text_content
0,53.png,182,360,582,391,44 50 71 269 2140 46
1,13.png,226,467,719,500,45 19 72 065 0203 50
2,29.png,241,374,642,404,41 16 71 364 1950 18
3,57.png,165,361,563,390,44 50 71 269 2142 16
4,15.png,199,466,692,501,45 19 72 065 0204 18


In [62]:
# Create the subsets and save them
train = ann[0:42]
train.to_csv('L:/DATA/ISIS/keras_ocr/train/annotations/train_annotations.csv',index=False)

val = ann[42:51]
val.to_csv('L:/DATA/ISIS/keras_ocr/val/annotations/val_annotations.csv',index=False)

test = ann[51:60]
test.to_csv('L:/DATA/ISIS/keras_ocr/test/annotations/test_annotations.csv',index=False)     

In [63]:
import shutil
import os

# Initialise the directories for training, validation and testing (remove current images)
train_dir = 'L:/DATA/ISIS/keras_ocr/train/'
val_dir = 'L:/DATA/ISIS/keras_ocr/val/'
test_dir = 'L:/DATA/ISIS/keras_ocr/test/'

for path in [train_dir, val_dir, test_dir]:
    for f in os.listdir(path+'images/'):
        os.remove(os.path.join(path, f))

all_images = os.listdir('L:/DATA/ISIS/keras_ocr/all images')

# copy images for all subsets
for image in allImages :
    full_name = os.path.join('L:/DATA/ISIS/keras_ocr/all images/', image)
    if image in train['image_path'].to_list():
        shutil.copy(full_name, train_dir+'images/'+image)
    elif image in val['image_path'].to_list():
        shutil.copy(full_name, val_dir+'images/'+image)
    elif image in test['image_path'].to_list():
        shutil.copy(full_name, test_dir+'images/'+image)

## 3. Create data generators

In [65]:
import numpy as np
import cv2
from tensorflow.keras.utils import Sequence